In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sbn 
import matplotlib.pyplot as plt 
import xgboost as xg 


In [19]:
dataset = pd.read_csv("Synthetic_Financial_datasets_log.csv")
dataset = dataset.drop(columns=['nameDest', 'nameOrig'])
dataset

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.00,160296.36,0.00,0.00,0,0
1,1,PAYMENT,1864.28,21249.00,19384.72,0.00,0.00,0,0
2,1,TRANSFER,181.00,181.00,0.00,0.00,0.00,1,0
3,1,CASH_OUT,181.00,181.00,0.00,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,41554.00,29885.86,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,339682.13,0.00,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,6311409.28,0.00,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,6311409.28,0.00,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,850002.52,0.00,0.00,0.00,1,0


In [24]:
dataset.info()

print(dataset.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   oldbalanceOrg   float64
 4   newbalanceOrig  float64
 5   oldbalanceDest  float64
 6   newbalanceDest  float64
 7   isFraud         int64  
 8   isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 436.9+ MB
Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud'],
      dtype='object')


In [18]:
dataset.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [30]:
for col in dataset.columns:
    if dataset[col].dtype == 'float64':
        dataset[col] = dataset[col].astype('int64')

In [31]:
# import polars 
# pl_data = polars.read_csv('Synthetic_Financial_datasets_log.csv')

# pl_data

In [32]:
# pl_data.columns

In [33]:
# pl_data.drop(columns=['nameOrig', 'nameDest', ])


In [34]:
dataset.isnull().sum()

step              0
type              0
amount            0
oldbalanceOrg     0
newbalanceOrig    0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [35]:
dataset['isFraud'].value_counts()

isFraud
0    6354407
1       8213
Name: count, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

dataset['type'] = le.fit_transform(dataset['type'])

x = dataset.drop(columns=['isFraud'])
y = dataset['isFraud']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

print(x.shape)
print('----')
print(dataset.info())

(6362620, 8)
----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 9 columns):
 #   Column          Dtype
---  ------          -----
 0   step            int64
 1   type            int64
 2   amount          int64
 3   oldbalanceOrg   int64
 4   newbalanceOrig  int64
 5   oldbalanceDest  int64
 6   newbalanceDest  int64
 7   isFraud         int64
 8   isFlaggedFraud  int64
dtypes: int64(9)
memory usage: 436.9 MB
None


In [40]:
fraud_model = xg.XGBClassifier(
    objective='multi:softprob',
    use_label_encoder=True,
    n_estimators=100,
    max_depth=7,
    learning_rate=0.1,
    num_class=2
)

fraud_model.fit(x_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=2,
              num_parallel_tree=None, ...)

In [42]:
test_accuracy = 100 * fraud_model.score(x_test, y_test)

print(f'The model has an accuracy of {test_accuracy:.2f}%')

ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [ ]:
from sklearn.metrics import classification_report

preds = fraud_model.predict(x_test)

print(classification_report(y_test, preds))

In [ ]:
from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(fraud_model, x_test, y_test, cv=5)

cv_score